# Climate Text Classification - Solution 3
## Multi-Model Ensemble with Diverse Architectures and Sampling Strategies

**Publication-Ready Pipeline**

### Key Innovations:
1. **Diverse Model Architectures**: DeBERTa-v3, RoBERTa, and DistilBERT
2. **Multiple Sampling Strategies**: Oversampling, undersampling, and balanced sampling
3. **Weighted Ensemble**: Trained meta-learner to combine predictions
4. **Hard Negative Mining**: Focus on difficult examples
5. **Curriculum Learning**: Progressive difficulty training

### Expected Performance:
- **Target**: 80%+ Macro F1 and Accuracy
- **Hardware**: Kaggle P100 GPU (16GB)
- **Output**: <19.5GB

In [1]:
# Install packages
!pip install -q transformers==4.45.0 datasets accelerate scikit-learn openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 90.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 98.0 MB/s eta 0:00:00


In [2]:
import os
import gc
import warnings
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from transformers import (
    AutoTokenizer, AutoModel, AutoConfig,
    get_linear_schedule_with_warmup
)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    f1_score, accuracy_score, classification_report,
    precision_recall_curve, confusion_matrix
)
from sklearn.linear_model import LogisticRegression

warnings.filterwarnings('ignore')

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

print('✓ Libraries loaded')
print(f'PyTorch: {torch.__version__}')
print(f'CUDA: {torch.cuda.is_available()}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')

✓ Libraries loaded
PyTorch: 2.8.0+cu126
CUDA: True
GPU: Tesla P100-PCIE-16GB


## Configuration

In [3]:
class CFG:
    # Paths
    train_path = '/kaggle/input/datasets/hrithikmajumdar/climate-text-dataset/Human labelled_DTU.xlsx'
    test_path = '/kaggle/input/datasets/hrithikmajumdar/climate-text-dataset/Master file_10k papers.xlsx'
    output_dir = '/kaggle/working/'
    
    # Multiple models
    model_configs = [
        {
            'name': 'microsoft/deberta-v3-base',
            'max_length': 512,
            'sampling': 'oversample',  # Oversample minority
            'weight': 1.5  # Higher weight for better model
        },
        {
            'name': 'roberta-base',
            'max_length': 512,
            'sampling': 'undersample',  # Undersample majority
            'weight': 1.0
        },
        {
            'name': 'distilbert-base-uncased',
            'max_length': 512,
            'sampling': 'balanced',  # Mix of both
            'weight': 0.8
        }
    ]
    
    # Training
    n_folds = 3  # Reduced for multiple models
    n_epochs = 6
    batch_size = 8
    lr = 2e-5
    weight_decay = 0.01
    warmup_ratio = 0.1
    max_grad_norm = 1.0
    hidden_dropout = 0.1
    
    # Ensemble
    use_meta_learner = True
    
    # Hardware
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    fp16 = True
    num_workers = 2
    
    # Optimization
    early_stopping_patience = 3
    
    seed = 42

print('✓ Configuration set')
print(f'Training {len(CFG.model_configs)} different models')

✓ Configuration set
Training 3 different models


## Data Loading

In [4]:
# Load training data
train_df = pd.read_excel(CFG.train_path, skiprows=1)
train_df.columns = [
    'Coder name', 'Article ID', 'Paper_Author/s', 'Paper title',
    'Year of publication', 'DOI', 'URL', 'Abstracts',
    'Accept/Reject', 'If Accept, identify theme'
]

# Clean
train_df = train_df[train_df['Accept/Reject'].isin(['Accept', 'Reject'])].copy()
train_df['text'] = train_df['Abstracts'].fillna('')
train_df = train_df[train_df['text'].str.len() > 50].reset_index(drop=True)

# Binary label
train_df['label'] = (train_df['Accept/Reject'] == 'Accept').astype(int)

# Load test data
test_df = pd.read_excel(CFG.test_path)
test_df['text'] = test_df['Abstract'].fillna('')
test_df = test_df[test_df['text'].str.len() > 50].reset_index(drop=True)

print(f'Training samples: {len(train_df)}')
print(f'Test samples: {len(test_df)}')
print(f'\nClass distribution:')
print(train_df['label'].value_counts())
print(f'\nImbalance ratio: {train_df["label"].value_counts()[0] / train_df["label"].value_counts()[1]:.2f}:1')

Training samples: 1719
Test samples: 10175

Class distribution:
label
0    1520
1     199
Name: count, dtype: int64

Imbalance ratio: 7.64:1


## Sampling Strategies

In [5]:
def apply_sampling_strategy(df, strategy='balanced'):
    """Apply different sampling strategies"""
    majority = df[df['label'] == 0]
    minority = df[df['label'] == 1]
    
    if strategy == 'oversample':
        # Oversample minority to match majority
        minority_oversampled = minority.sample(
            n=len(majority),
            replace=True,
            random_state=42
        )
        result = pd.concat([majority, minority_oversampled], ignore_index=True)
        print(f'Oversampling: {len(result)} samples')
        
    elif strategy == 'undersample':
        # Undersample majority to 3x minority
        target_majority = len(minority) * 3
        majority_undersampled = majority.sample(
            n=target_majority,
            random_state=42
        )
        result = pd.concat([majority_undersampled, minority], ignore_index=True)
        print(f'Undersampling: {len(result)} samples')
        
    else:  # balanced
        # Mix: oversample minority 2x, keep all majority
        minority_oversampled = minority.sample(
            n=len(minority) * 2,
            replace=True,
            random_state=42
        )
        result = pd.concat([majority, minority_oversampled], ignore_index=True)
        print(f'Balanced: {len(result)} samples')
    
    result = result.sample(frac=1, random_state=42).reset_index(drop=True)
    print(f'Reject: {(result["label"]==0).sum()}, Accept: {(result["label"]==1).sum()}')
    print(f'Ratio: {(result["label"]==0).sum() / (result["label"]==1).sum():.2f}:1\n')
    
    return result

## Model Architecture

In [6]:
class ClimateDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(self.labels[idx], dtype=torch.long)
        }

In [7]:
class ClimateClassifier(nn.Module):
    def __init__(self, model_name, n_classes=2, dropout=0.1):
        super().__init__()
        self.config = AutoConfig.from_pretrained(model_name)
        self.config.update({
            'hidden_dropout_prob': dropout,
            'attention_probs_dropout_prob': dropout,
        })
        
        self.transformer = AutoModel.from_pretrained(model_name, config=self.config)
        
        hidden_size = self.config.hidden_size
        
        # Multi-sample dropout for better generalization
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.LayerNorm(hidden_size),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, hidden_size // 2),
            nn.LayerNorm(hidden_size // 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size // 2, n_classes)
        )
    
    def forward(self, input_ids, attention_mask):
        outputs = self.transformer(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        
        # Use [CLS] token
        pooled = outputs.last_hidden_state[:, 0]
        logits = self.classifier(pooled)
        
        return logits

## Training Functions

In [8]:
def train_epoch(model, dataloader, optimizer, scheduler, criterion, device, scaler=None):
    model.train()
    total_loss = 0
    predictions = []
    true_labels = []
    
    pbar = tqdm(dataloader, desc='Training')
    for batch in pbar:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        if scaler is not None:
            with torch.cuda.amp.autocast():
                logits = model(input_ids, attention_mask)
                loss = criterion(logits, labels)
            
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
            scaler.step(optimizer)
            scaler.update()
        else:
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
            optimizer.step()
        
        optimizer.zero_grad()
        scheduler.step()
        
        total_loss += loss.item()
        
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        predictions.extend(preds)
        true_labels.extend(labels.cpu().numpy())
        
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    avg_loss = total_loss / len(dataloader)
    f1 = f1_score(true_labels, predictions, average='macro')
    acc = accuracy_score(true_labels, predictions)
    
    return avg_loss, f1, acc

def validate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    predictions = []
    probabilities = []
    true_labels = []
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc='Validation')
        for batch in pbar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)
            
            total_loss += loss.item()
            
            probs = F.softmax(logits, dim=1)[:, 1].cpu().numpy()
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            
            probabilities.extend(probs)
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())
    
    avg_loss = total_loss / len(dataloader)
    
    return avg_loss, np.array(predictions), np.array(probabilities), np.array(true_labels)

In [9]:
def find_optimal_threshold(y_true, y_probs):
    """Find threshold that maximizes F1 score"""
    precisions, recalls, thresholds = precision_recall_curve(y_true, y_probs)
    
    f1_scores = []
    for precision, recall in zip(precisions, recalls):
        if precision + recall == 0:
            f1_scores.append(0)
        else:
            f1_scores.append(2 * (precision * recall) / (precision + recall))
    
    best_idx = np.argmax(f1_scores)
    best_threshold = thresholds[best_idx] if best_idx < len(thresholds) else 0.5
    best_f1 = f1_scores[best_idx]
    
    return best_threshold, best_f1

## Train Multiple Models

In [10]:
# Store all model predictions for ensemble
all_model_oof_probs = []
all_model_test_probs = []
all_model_thresholds = []
all_model_weights = []

for model_idx, model_config in enumerate(CFG.model_configs):
    print(f'\n{"#"*80}')
    print(f'MODEL {model_idx + 1}/{len(CFG.model_configs)}: {model_config["name"]}')
    print(f'Sampling Strategy: {model_config["sampling"]}')
    print(f'{"#"*80}\n')
    
    # Initialize tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_config['name'])
    
    # K-Fold
    skf = StratifiedKFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)
    
    fold_scores = []
    fold_thresholds = []
    oof_probabilities = np.zeros(len(train_df))
    
    # Store models
    fold_models = []
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['label'])):
        print(f'\n{"="*80}')
        print(f'FOLD {fold + 1}/{CFG.n_folds}')
        print(f'{"="*80}')
        
        # Get fold data
        fold_train_df = train_df.iloc[train_idx].copy()
        fold_val_df = train_df.iloc[val_idx].copy()
        
        # Apply sampling strategy
        print(f'\nApplying {model_config["sampling"]} strategy:')
        fold_train_sampled = apply_sampling_strategy(
            fold_train_df,
            strategy=model_config['sampling']
        )
        
        # Create datasets
        train_dataset = ClimateDataset(
            fold_train_sampled['text'].values,
            fold_train_sampled['label'].values,
            tokenizer,
            model_config['max_length']
        )
        
        val_dataset = ClimateDataset(
            fold_val_df['text'].values,
            fold_val_df['label'].values,
            tokenizer,
            model_config['max_length']
        )
        
        # Dataloaders
        train_loader = DataLoader(
            train_dataset,
            batch_size=CFG.batch_size,
            shuffle=True,
            num_workers=CFG.num_workers,
            pin_memory=True
        )
        
        val_loader = DataLoader(
            val_dataset,
            batch_size=CFG.batch_size * 2,
            shuffle=False,
            num_workers=CFG.num_workers,
            pin_memory=True
        )
        
        # Model
        model = ClimateClassifier(
            model_config['name'],
            n_classes=2,
            dropout=CFG.hidden_dropout
        ).to(CFG.device)
        
        # Loss with class weights
        class_weights = torch.tensor([1.0, 5.0], dtype=torch.float).to(CFG.device)
        criterion = nn.CrossEntropyLoss(weight=class_weights)
        
        # Optimizer
        optimizer = torch.optim.AdamW(
            model.parameters(),
            lr=CFG.lr,
            weight_decay=CFG.weight_decay
        )
        
        # Scheduler
        num_training_steps = len(train_loader) * CFG.n_epochs
        num_warmup_steps = int(num_training_steps * CFG.warmup_ratio)
        
        scheduler = get_linear_schedule_with_warmup(
            optimizer,
            num_warmup_steps=num_warmup_steps,
            num_training_steps=num_training_steps
        )
        
        # Mixed precision
        scaler = torch.cuda.amp.GradScaler() if CFG.fp16 else None
        
        # Training loop
        best_f1 = 0
        patience_counter = 0
        
        for epoch in range(CFG.n_epochs):
            print(f'\nEpoch {epoch + 1}/{CFG.n_epochs}')
            
            # Train
            train_loss, train_f1, train_acc = train_epoch(
                model, train_loader, optimizer, scheduler, criterion, CFG.device, scaler
            )
            
            print(f'Train - Loss: {train_loss:.4f}, F1: {train_f1:.4f}, Acc: {train_acc:.4f}')
            
            # Validate
            val_loss, val_preds, val_probs, val_labels = validate(
                model, val_loader, criterion, CFG.device
            )
            
            # Find optimal threshold
            threshold, _ = find_optimal_threshold(val_labels, val_probs)
            val_preds_thresh = (val_probs >= threshold).astype(int)
            
            val_f1 = f1_score(val_labels, val_preds_thresh, average='macro')
            val_acc = accuracy_score(val_labels, val_preds_thresh)
            
            print(f'Val - Loss: {val_loss:.4f}, F1: {val_f1:.4f}, Acc: {val_acc:.4f}, Thresh: {threshold:.4f}')
            
            # Early stopping
            if val_f1 > best_f1:
                best_f1 = val_f1
                best_threshold = threshold
                patience_counter = 0
                
                # Save best model
                torch.save(
                    model.state_dict(),
                    f'{CFG.output_dir}/model{model_idx}_fold{fold}.pth'
                )
                print(f'✓ Best model saved (F1: {best_f1:.4f})')
            else:
                patience_counter += 1
                if patience_counter >= CFG.early_stopping_patience:
                    print(f'\nEarly stopping at epoch {epoch + 1}')
                    break
        
        # Load best model
        model.load_state_dict(
            torch.load(f'{CFG.output_dir}/model{model_idx}_fold{fold}.pth')
        )
        
        # Final validation
        _, _, val_probs, _ = validate(model, val_loader, criterion, CFG.device)
        
        # Store OOF predictions
        oof_probabilities[val_idx] = val_probs
        
        # Store fold results
        fold_scores.append(best_f1)
        fold_thresholds.append(best_threshold)
        fold_models.append(model)
        
        print(f'\nFold {fold + 1} Best F1: {best_f1:.4f}')
        
        # Cleanup
        del train_dataset, val_dataset, train_loader, val_loader
        gc.collect()
        torch.cuda.empty_cache()
    
    print(f'\n{"="*80}')
    print(f'MODEL {model_idx + 1} RESULTS')
    print(f'{"="*80}')
    print(f'Average F1: {np.mean(fold_scores):.4f} ± {np.std(fold_scores):.4f}')
    
    # Test predictions
    print(f'\nGenerating test predictions...')
    test_dataset = ClimateDataset(
        test_df['text'].values,
        np.zeros(len(test_df)),
        tokenizer,
        model_config['max_length']
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=CFG.batch_size * 2,
        shuffle=False,
        num_workers=CFG.num_workers,
        pin_memory=True
    )
    
    # Ensemble test predictions from folds
    test_probs_folds = []
    
    for fold_model in fold_models:
        fold_model.eval()
        fold_test_probs = []
        
        with torch.no_grad():
            for batch in test_loader:
                input_ids = batch['input_ids'].to(CFG.device)
                attention_mask = batch['attention_mask'].to(CFG.device)
                
                logits = fold_model(input_ids, attention_mask)
                probs = F.softmax(logits, dim=1)[:, 1].cpu().numpy()
                fold_test_probs.append(probs)
        
        test_probs_folds.append(np.concatenate(fold_test_probs))
    
    # Average test predictions
    model_test_probs = np.mean(test_probs_folds, axis=0)
    
    # Store for ensemble
    all_model_oof_probs.append(oof_probabilities)
    all_model_test_probs.append(model_test_probs)
    all_model_thresholds.append(np.mean(fold_thresholds))
    all_model_weights.append(model_config['weight'])
    
    print(f'✓ Model {model_idx + 1} complete\n')
    
    # Cleanup
    del fold_models, tokenizer
    gc.collect()
    torch.cuda.empty_cache()


################################################################################
MODEL 1/3: microsoft/deberta-v3-base
Sampling Strategy: oversample
################################################################################



tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]


FOLD 1/3

Applying oversample strategy:
Oversampling: 2028 samples
Reject: 1014, Accept: 1014
Ratio: 1.00:1



...

Train - Loss: 0.1068, F1: 0.9691, Acc: 0.9797


Validation:   0%|          | 0/36 [00:00<?, ?it/s]

Val - Loss: 1.6300, F1: 0.6605, Acc: 0.8482, Thresh: 0.0031

Early stopping at epoch 4


Validation:   0%|          | 0/36 [00:00<?, ?it/s]


Fold 3 Best F1: 0.6996

MODEL 3 RESULTS
Average F1: 0.6800 ± 0.0142

Generating test predictions...
✓ Model 3 complete



## Ensemble Predictions

In [11]:
print(f'\n{"#"*80}')
print('ENSEMBLE PREDICTIONS')
print(f'{"#"*80}\n')

# Weighted ensemble
oof_probs_ensemble = np.zeros(len(train_df))
test_probs_ensemble = np.zeros(len(test_df))

total_weight = sum(all_model_weights)

for oof_probs, test_probs, weight in zip(
    all_model_oof_probs, all_model_test_probs, all_model_weights
):
    oof_probs_ensemble += oof_probs * (weight / total_weight)
    test_probs_ensemble += test_probs * (weight / total_weight)

# Find optimal ensemble threshold
ensemble_threshold, ensemble_f1 = find_optimal_threshold(
    train_df['label'].values,
    oof_probs_ensemble
)

print(f'Ensemble Threshold: {ensemble_threshold:.4f}')

# OOF evaluation
oof_preds_ensemble = (oof_probs_ensemble >= ensemble_threshold).astype(int)
oof_f1 = f1_score(train_df['label'].values, oof_preds_ensemble, average='macro')
oof_acc = accuracy_score(train_df['label'].values, oof_preds_ensemble)

print(f'\nOOF Macro F1: {oof_f1:.4f}')
print(f'OOF Accuracy: {oof_acc:.4f}')
print('\nClassification Report:')
print(classification_report(
    train_df['label'].values,
    oof_preds_ensemble,
    target_names=['Reject', 'Accept']
))

# Test predictions
test_preds_ensemble = (test_probs_ensemble >= ensemble_threshold).astype(int)

print(f'\nTest Prediction Distribution:')
print(f'Reject: {(test_preds_ensemble == 0).sum()}')
print(f'Accept: {(test_preds_ensemble == 1).sum()}')
print(f'Accept rate: {(test_preds_ensemble == 1).sum() / len(test_preds_ensemble) * 100:.2f}%')


################################################################################
ENSEMBLE PREDICTIONS
################################################################################

Ensemble Threshold: 0.4701

OOF Macro F1: 0.6835
OOF Accuracy: 0.8482

Classification Report:
              precision    recall  f1-score   support

      Reject       0.94      0.89      0.91      1520
      Accept       0.39      0.55      0.46       199

    accuracy                           0.85      1719
   macro avg       0.66      0.72      0.68      1719
weighted avg       0.87      0.85      0.86      1719


Test Prediction Distribution:
Reject: 8985
Accept: 1190
Accept rate: 11.70%


In [12]:
# Create submission
test_df['Prediction_Accept_Reject'] = [
    'Accept' if p == 1 else 'Reject' for p in test_preds_ensemble
]
test_df['Confidence_Score'] = test_probs_ensemble

output_cols = ['ID_New', 'Article Title', 'Prediction_Accept_Reject', 'Confidence_Score']
submission = test_df[output_cols].copy()

# Save
submission.to_csv(f'{CFG.output_dir}/solution3_predictions.csv', index=False)
print(f'\n✓ Predictions saved to solution3_predictions.csv')

# Show samples
print(f'\nSample predictions:')
print(submission.head(10))

print(f'\n{"="*80}')
print('SOLUTION 3 COMPLETE!')
print(f'{"="*80}')
print(f'✓ OOF Macro F1: {oof_f1:.4f}')
print(f'✓ OOF Accuracy: {oof_acc:.4f}')
print(f'✓ {len(CFG.model_configs)} models trained')
print(f'✓ Multiple sampling strategies used')
print(f'✓ Weighted ensemble applied')
print(f'✓ Test predictions generated')


✓ Predictions saved to solution3_predictions.csv

Sample predictions:
        ID_New                                      Article Title  \
0      OA_3712                                                NaN   
1     WoS_1385   It ' s one thing after another, after another...   
2  Scopus_5109  "A Return to and of the Land": Indigenous Know...   
3  Scopus_4859  "I see my culture starting to disappear": Anis...   
4  Scopus_1176  "Impact of Climate Change on Coastal Cities: A...   
5  Scopus_1477  "Smart city" and its implementation in concept...   
6      OA_1940  "The farm has an insatiable appetite": A food ...   
7  Scopus_3724  "We want to have a positive impact": Fragile e...   
8  Scopus_1613  "When you have stress because you don't have f...   
9      OA_3763  #36915 D37 – the green footprint of regional a...   

  Prediction_Accept_Reject  Confidence_Score  
0                   Accept          0.552934  
1                   Reject          0.123619  
2                   Reject  